<a href="https://colab.research.google.com/github/paologastaldi-polito/siren/blob/master/src/explore_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment initialization

In [ ]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip3 install 'tensorflow'

# Project initialization

###Install requirements

In [ ]:
!pip3 install 'sk-video'
!pip3 install 'cmapy'
!pip3 install 'wget'
!pip3 install 'python-box'

###Reset filesystem

In [ ]:
# !rm -rf 'SRGAN-PyTorch'
# !rm -rf 'siren'
# !rm -rf 'BSDS500'
# !rm -rf 'DIV2K-loader'
# !rm -rf 'data'

### Import libraries

In [ ]:
### IMPORTANT: due to some file conflicts, import first the SRGAN code, then the SIREN one ###

import sys, os

# Clone SRGAN github repository and import the model
if not os.path.isdir('SRGAN-PyTorch'):
    !git clone 'https://github.com/twhui/SRGAN-PyTorch.git'

sys.path.append('SRGAN-PyTorch')
import models, options, utils

# Clone SIREN github repository and import the model
# if not os.path.isdir('siren'):
#     !git clone 'https://github.com/paologastaldi-polito/siren.git'
!rm -rf 'siren'
!git clone 'https://github.com/paologastaldi-polito/siren.git'

sys.path.append('siren/src/mylibs')
import myutils, mydataloader, siren, relu, loss_functions
from mydataloader import PoissonEqn
from siren import Siren
from relu import Relu

# import useful libraries
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import time
import torch.nn.init as init
from functools import partial
from tqdm import tqdm

### Import datasets

In [ ]:
# Import repository with the BDDS500 dataset
if not os.path.isdir('BSDS500'):
    !git clone 'https://github.com/BIDS/BSDS500.git'

# Import repository with the DIV2K dataloader
if not os.path.isdir('DIV2K-loader'):
    !git clone 'https://github.com/paologastaldi-polito/DIV2K-loader.git'
    !mv 'DIV2K-loader/DIV2K-loader.py' 'DIV2K-loader/DIV2K_loader.py'

sys.path.append('DIV2K-loader')
import DIV2K_loader
from DIV2K_loader import DIV2KImport, DIV2KImageDataset, default_transform

#1 - Implement the network

In [ ]:
HIDDEN_LAYERS = 2
LR = 1e-4
TOTAL_STEPS = 500 # for testing the code
# TOTAL_STEPS = 15000 # for real training
STEPS_TIL_SUMMARY = 500
VERBOSE = False
sidelength = 256

#2 - Image fitting and Poisson image reconstruction

###2.1 - Image fitting

#####2.1.1 - Prepare data

In [ ]:
dataset = PoissonEqn(sidelength)
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(dataloader))
gt_cameraman = gt
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

#####2.1.2 - Train on image

In [ ]:
# Create the network
net_type = 'siren'
training = 'on_image'
net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256) # IMPORTANT: 3 layers to compare with SIREN paper results
model = net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output, coords = model(model_input)    
    loss = loss_functions.image_mse(model_output, coords, gt['img'])
    psnr = myutils.psnr(model_output, gt['img'])
    psnrs.append(psnr)
    ssim = myutils.ssim(model_output, gt['img'])
    ssims.append(ssim)

    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            img = model_output.clone()
            img_grads = myutils.gradient(model_output, coords)
            img_laplacian = myutils.laplace(model_output, coords)

            img_dict = {
                'img' : img,
                'grads' : img_grads,
                'laplace' : img_laplacian
            }
            if step == total_steps:
                fname = net_type + '_' + str(HIDDEN_LAYERS) + '_layers_' + str(total_steps) + '_' + training + '_' + str(sidelength) + '.png'
                myutils.plot_all(img_dict, gt, sidelength=sidelength, save=True, fname=fname)
            else:
                myutils.plot_all(img_dict, gt, sidelength=sidelength)

    optim.zero_grad()
    loss.backward()
    optim.step()

print('Ground truth')
myutils.plot_all(gt, gt, sidelength=sidelength)

siren_image = {key: value.cpu().detach() for key, value in img_dict.items()}
psnr_siren_image = psnrs
ssims_siren_image = ssims

###2.2 - Poisson image reconstruction

#####2.2.1 - Prepare data

In [ ]:
dataset = PoissonEqn(sidelength, img_type='starfish')
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(dataloader))
gt['grads'] *= 10
gt['laplace'] *= 1e4
gt_starfish = gt
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

#####2.2.2 - Train on gradient

In [ ]:
# Create the network
net_type = 'siren'
training = 'on_gradient'
net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256) # IMPORTANT: 3 layers to compare with SIREN paper results
model = net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output, coords = model(model_input)    
    loss = loss_functions.gradients_mse(model_output, coords, gt['grads'])
    # psnr = myutils.psnr(model_output, gt['img'])
    # psnrs.append(psnr)
    # ssim = myutils.ssim(model_output, gt['img'])
    # ssims.append(ssim)

    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))

        if verbose or step == total_steps:
            img = model_output.clone()
            img_grads = myutils.gradient(model_output, coords)
            img_laplacian = myutils.laplace(model_output, coords)

            img_dict = {
                'img' : img,
                'grads' : img_grads,
                'laplace' : img_laplacian
            }
            if step == total_steps:
                fname = net_type + '_' + str(HIDDEN_LAYERS) + '_layers_' + str(total_steps) + '_' + training + '_' + str(sidelength) + '.png'
                myutils.plot_all(img_dict, gt, sidelength=sidelength, save=True, fname=fname)
            else:
                myutils.plot_all(img_dict, gt, sidelength=sidelength)

    optim.zero_grad()
    loss.backward()
    optim.step()

print('Ground truth')
myutils.plot_all(gt, gt, sidelength=sidelength)

siren_grad = {key: value.cpu().detach() for key, value in img_dict.items()}
psnr_siren_grad = psnrs
ssims_siren_grad = ssims

#####2.2.3 - Train on laplacian

In [ ]:
# Create the network
net_type = 'siren'
training = 'on_laplace'
net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256) # IMPORTANT: 3 layers to compare with SIREN paper results
model = net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output, coords = model(model_input)    
    loss = loss_functions.laplace_mse(model_output, coords, gt['laplace'])
    # psnr = myutils.psnr(model_output, gt['img'])
    # psnrs.append(psnr)
    # ssim = myutils.ssim(model_output, gt['img'])
    # ssims.append(ssim)

    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        if verbose or step == total_steps:
            img = model_output.clone()
            img_grads = myutils.gradient(model_output, coords)
            img_laplacian = myutils.laplace(model_output, coords)

            img_dict = {
                'img' : img,
                'grads' : img_grads,
                'laplace' : img_laplacian
            }
            if step == total_steps:
                fname = net_type + '_' + str(HIDDEN_LAYERS) + '_layers_' + str(total_steps) + '_' + training + '_' + str(sidelength) + '.png'
                myutils.plot_all(img_dict, gt, sidelength=sidelength, save=True, fname=fname)
            else:
                myutils.plot_all(img_dict, gt, sidelength=sidelength)

    optim.zero_grad()
    loss.backward()
    optim.step()


print('Ground truth')
myutils.plot_all(gt, gt, sidelength=sidelength)

siren_lapl = {key: value.cpu().detach() for key, value in img_dict.items()}
psnr_siren_lapl = psnrs
ssims_siren_lapl = ssims

#3 - Compare with ReLU

###3.1 - Prepare data

In [ ]:
dataset = PoissonEqn(sidelength)
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(dataloader))
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

###3.2 - Train on image

In [ ]:
# Create the network
net_type = 'relu'
training = 'on_image'
net = Relu(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256) # IMPORTANT: 3 layers to compare with SIREN paper results
model = net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output, coords = model(model_input)    
    loss = loss_functions.image_mse(model_output, coords, gt['img'])
    psnr = myutils.psnr(model_output, gt['img'])
    psnrs.append(psnr)
    ssim = myutils.ssim(model_output, gt['img'])
    ssims.append(ssim)

    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            img = model_output.clone()
            img_grads = myutils.gradient(model_output, coords)
            img_laplacian = myutils.laplace(model_output, coords)

            img_dict = {
                'img' : img,
                'grads' : img_grads,
                'laplace' : img_laplacian
            }
            if step == total_steps:
                fname = net_type + '_' + str(HIDDEN_LAYERS) + '_layers_' + str(total_steps) + '_' + training + '_' + str(sidelength) + '.png'
                myutils.plot_all(img_dict, gt, sidelength=sidelength, save=True, fname=fname)
            else:
                myutils.plot_all(img_dict, gt, sidelength=sidelength)

    optim.zero_grad()
    loss.backward()
    optim.step()

print('Ground truth')
myutils.plot_all(gt, gt, sidelength=sidelength)

relu_image = {key: value.cpu().detach() for key, value in img_dict.items()}
psnr_relu_image = psnrs
ssims_relu_image = ssims

###3.3 - Prepare data

In [ ]:
dataset = PoissonEqn(sidelength, img_type='starfish')
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(dataloader))
gt['grads'] *= 10
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

###3.4 - Train on gradient

In [ ]:
# Create the network
net_type = 'relu'
training = 'on_gradient'
net = Relu(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256) # IMPORTANT: 3 layers to compare with SIREN paper results
model = net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output, coords = model(model_input)    
    loss = loss_functions.gradients_mse(model_output, coords, gt['grads'])
    # psnr = myutils.psnr(model_output, gt['img'])
    # psnrs.append(psnr)
    # ssim = myutils.ssim(model_output, gt['img'])
    # ssims.append(ssim)

    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))

        if verbose or step == total_steps:
            img = model_output.clone()
            img_grads = myutils.gradient(model_output, coords)
            img_laplacian = myutils.laplace(model_output, coords)

            img_dict = {
                'img' : img,
                'grads' : img_grads,
                'laplace' : img_laplacian
            }
            if step == total_steps:
                fname = net_type + '_' + str(HIDDEN_LAYERS) + '_layers_' + str(total_steps) + '_' + training + '_' + str(sidelength) + '.png'
                myutils.plot_all(img_dict, gt, sidelength=sidelength, save=True, fname=fname)
            else:
                myutils.plot_all(img_dict, gt, sidelength=sidelength)

    optim.zero_grad()
    loss.backward()
    optim.step()

print('Ground truth')
myutils.plot_all(gt, gt, sidelength=sidelength)

relu_grad = {key: value.cpu().detach() for key, value in img_dict.items()}
psnr_relu_grad = psnrs
ssims_relu_grad = ssims

#Collect and compare results

In [ ]:
# SIREN vs ReLU PSNR
psnr_data = {'SIREN' : psnr_siren_image, 'ReLU' : psnr_relu_image}
myutils.plot_psnrs(psnr_data, TOTAL_STEPS, 'SIREN vs ReLU', save=True, fname='psnr_siren_vs_relu_2_layers_15000_on_image_256.png')

# (GT, SIREN, ReLU) x (image, gradient, laplace)
myutils.print_fitting_grid(gt_cameraman, siren_image, relu_image, psnr_siren_image[-1], psnr_relu_image[-1], sidelength=sidelength)

# (GT, GRAD(ReLU), GRAD(SIREN), LAPL(SIREN)) x (image, gradient, laplace)
myutils.print_poisson_grid(gt_starfish, relu_grad, siren_grad, siren_lapl, sidelength=sidelength)

#4 - Ablation studies

###4.1 - Weights initialization

####4.1.1 - change weights of the first layer

#####4.1.1.1 - Prepare data

In [ ]:
dataset = PoissonEqn(256, gradients=True, laplace=True)
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(dataloader))
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

#####4.1.1.2 - PSNR evaluation

In [ ]:
psnr_data = {}

net_type = 'siren'

# Weight initialization of first layer
init_weights = [[None, "standard weights"],
                [partial(init.kaiming_uniform_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Uniform"],
                [partial(init.kaiming_normal_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Normal"],
                [partial(init.uniform_, a=0.0, b=1.0), "Uniform"],
                [partial(init.normal_, mean=0.0, std=1.0), "Normal"],
                [partial(init.xavier_uniform_, gain=1.0), "Xavier Uniform"],
                [partial(init.xavier_normal_, gain=1.0), "Xavier Normal"]
                ]

for weights in init_weights:
    print(weights[1])
    psnrs = []
    
    net = Siren(n_hidden_layers=3, hidden_features=256)
    
    if weights[0] is not None:
        weights[0](net.net[0].linear.weight)

    model = net.cuda()

    # Define optimizer
    optim = torch.optim.Adam(lr=1e-4, params=model.parameters())

    total_steps = 1000
    steps_til_summary = STEPS_TIL_SUMMARY
    verbose = VERBOSE # True, False

    for step in range(total_steps+1):
        model_output, coords = model(model_input)    
        loss = loss_functions.image_mse(model_output, coords, gt['img'])
        psnr = myutils.psnr(model_output, gt['img'])
        psnrs.append(psnr)
        if not step % steps_til_summary:
            print("Step %d, Total loss %0.6f" % (step, loss))
            if verbose or step == total_steps:
                img = model_output.clone()
                img_grads = myutils.sobel_filter(model_output.clone())
                img_laplacian = myutils.laplace_filter(model_output.clone())

                img_dict = {
                    'img' : img,
                    'grads' : img_grads,
                    'laplace' : img_laplacian
                }
                if step == total_steps:
                    fname = net_type + '_' + str(HIDDEN_LAYERS) + '_layers_' + str(total_steps) + '_' + weights[1] + '_' + str(sidelength) + '.png'
                    myutils.plot_all(img_dict, gt, sidelength=sidelength, save=True, fname=fname)
                else:
                    myutils.plot_all(img_dict, gt, sidelength=sidelength)

        optim.zero_grad()
        loss.backward()
        optim.step()

    psnr_data[weights[1]] = psnrs

myutils.plot_psnrs(psnr_data, total_steps, 'init weights', save=True, fname='psnr_siren_2_layers_2000_256_first_layer_initialization.png')

#####4.1.1.3 - Layer activation evaluation

In [ ]:
# Distribution configuration
distribution = None
# distribution = [partial(init.kaiming_uniform_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Uniform"]
# distribution = [partial(init.kaiming_normal_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Normal"]
# distribution = [partial(init.uniform_, a=0.0), "Uniform"]
# distribution = [partial(init.normal_, mean=0.0), "Normal"]
# distribution = [partial(init.xavier_uniform_, gain=1.0), "Xavier Uniform"]
# distribution = [partial(init.xavier_normal_, gain=1.0), "Xavier Normal"]

model = Siren(n_hidden_layers=HIDDEN_LAYERS, outermost_linear=True, with_activations=True) # with_activation = True so that the number of features is automatically set
if distribution is not None:
  distribution[0](model.net[0].linear.weight)
  title = "SIREN first layer initialized with " + distribution[1]
else:
  title = "SIREN first layer initialized with standard weights"
model = model.cuda()
activations = model.forward_with_activations(model_input) # Calling forward with activation in order to return activation to plot
myutils.plot_all_activations_and_grads(activations, title=title)

####4.1.2 - change weights of the hidden layers

#####4.1.2.1 - Prepare data

In [ ]:
dataset = PoissonEqn(256, gradients=True, laplace=True)
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(dataloader))
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

#####4.1.2.2 - PSNR evaluation

In [ ]:
psnr_data = {}

net_type = 'siren'
layers = 3

# Weight initialization of first layer
init_weights = [[None, "standard weights"],
                [partial(init.uniform_), "Uniform"],
                [partial(init.normal_), "Normal"],
                [partial(init.kaiming_uniform_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Uniform"],
                [partial(init.kaiming_normal_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Normal"],
                [partial(init.xavier_uniform_, gain=1.0), "Xavier Uniform"],
                [partial(init.xavier_normal_, gain=1.0), "Xavier Normal"]
                ]

for weights in init_weights:
    print(weights[1])
    psnrs = []
    
    net = Siren(n_hidden_layers=layers, hidden_features=256)

    if weights[0] is not None:
        for l in range(1, layers+1):
            weights[0](net.net[l].linear.weight)

    model = net.cuda()

    # Define optimizer
    optim = torch.optim.Adam(lr=1e-4, params=model.parameters())

    total_steps = 1000
    steps_til_summary = STEPS_TIL_SUMMARY
    verbose = VERBOSE # True, False

    for step in range(total_steps+1):
        model_output, coords = model(model_input)    
        loss = loss_functions.image_mse(model_output, coords, gt['img'])
        psnr = myutils.psnr(model_output, gt['img'])
        psnrs.append(psnr)
        if not step % steps_til_summary:
            print("Step %d, Total loss %0.6f" % (step, loss))
            if verbose or step == total_steps:
                img = model_output.clone()
                img_grads = myutils.sobel_filter(model_output.clone())
                img_laplacian = myutils.laplace_filter(model_output.clone())

                img_dict = {
                    'img' : img,
                    'grads' : img_grads,
                    'laplace' : img_laplacian
                }
                if step == total_steps:
                    fname = net_type + '_' + str(HIDDEN_LAYERS) + '_layers_' + str(total_steps) + '_' + weights[1] + '_' + str(sidelength) + '.png'
                    myutils.plot_all(img_dict, gt, sidelength=sidelength, save=True, fname=fname)
                else:
                    myutils.plot_all(img_dict, gt, sidelength=sidelength)

        optim.zero_grad()
        loss.backward()
        optim.step()

    psnr_data[weights[1]] = psnrs

myutils.plot_psnrs(psnr_data, total_steps, 'init weights', save=True, fname='psnr_siren_2_layers_1000_256_hidden_layer_initialization.png')

#####4.1.2.3 - Layer activation evaluation

In [ ]:
# Distribution configuration
# distribution = None
distribution = [partial(init.kaiming_uniform_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Uniform"]
# distribution = [partial(init.kaiming_normal_, a=0.0, nonlinearity='relu', mode='fan_in'), "Kaiming Normal"]
# distribution = [partial(init.uniform_, a=0.0), "Uniform"]
# distribution = [partial(init.normal_, mean=0.0), "Normal"]
# distribution = [partial(init.xavier_uniform_, gain=1.0), "Xavier Uniform"]
# distribution = [partial(init.xavier_normal_, gain=1.0), "Xavier Normal"]

model = Siren(n_hidden_layers=HIDDEN_LAYERS, outermost_linear=True, with_activations=True) # with_activation = True so that the number of features is automatically set
if distribution is not None:
  for l in range(1, HIDDEN_LAYERS+1):
    distribution[0](model.net[l].linear.weight)
  title = "SIREN hidden layers initialized with " + distribution[1]
else:
  title = "SIREN hidden layers initialized with standard weights"
model = model.cuda()
activations = model.forward_with_activations(model_input) # Calling forward with activation in order to return activation to plot
myutils.plot_all_activations_and_grads(activations, title=title)

###4.2 - ω0 initialization 

#####4.2.1 - Prepare data

In [ ]:
dataset = PoissonEqn(sidelength)
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(dataloader))
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

#####4.2.2 - change ω0 in first layer

In [ ]:
psnr_data = {}
first_omega_0 = [
                 8,
                 15,
                 30,
                 60,
                 120,
                 240
                ]

for w0 in first_omega_0:
    print("w0:", w0)
    # Create the network
    net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256, first_omega_0=w0, hidden_omega_0=30)
    model = net.cuda()

    # Define optimizer
    optim = torch.optim.Adam(lr=1e-4, params=model.parameters())
    total_steps = 1000
    steps_til_summary = 100

    psnrs = []

    for step in range(total_steps+1):
        model_output, coords = model(model_input)    
        loss = loss_functions.image_mse(model_output, coords, gt['img'])
        psnr = myutils.psnr(model_output, gt['img'])
        psnrs.append(psnr)
        if not step % steps_til_summary:
            print("Step %d, Total loss %0.6f" % (step, loss))

        optim.zero_grad()
        loss.backward()
        optim.step()

    psnr_data[str(w0)] = psnrs

myutils.plot_psnrs(psnr_data, total_steps, 'w0 in first layer', save=True, fname='psnr_w0_first_layer_1000_10_layers.png')

#####4.2.3 - change ω0 in hidden layers

In [ ]:
psnr_data = {}
hidden_omega_0 = [
                  8,
                  15,
                  30,
                  60,
                  120,
                  240
                  ]

for w0 in hidden_omega_0:
    print("w0:", w0)
    # Create the network
    net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256, first_omega_0=30, hidden_omega_0=w0)
    model = net.cuda()

    # Define optimizer
    optim = torch.optim.Adam(lr=1e-4, params=model.parameters())
    total_steps = 1000
    steps_til_summary = 100

    psnrs = []

    for step in range(total_steps+1):
        model_output, coords = model(model_input)    
        loss = loss_functions.image_mse(model_output, coords, gt['img'])
        psnr = myutils.psnr(model_output, gt['img'])
        psnrs.append(psnr)
        if not step % steps_til_summary:
            print("Step %d, Total loss %0.6f" % (step, loss))

        optim.zero_grad()
        loss.backward()
        optim.step()

    psnr_data[str(w0)] = psnrs

myutils.plot_psnrs(psnr_data, total_steps, 'w0 in hidden layers', save=True, fname='psnr_w0_hidden_layers_1000_2_layers.png')

#####4.2.4 - change ω0 in all the layers

In [ ]:
psnr_data = {}
omega_0 = [
          8,
          15,
          30,
          60,
          120,
          240
          ]

for w0 in omega_0:
    print("w0:", w0)
    # Create the network
    net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=256, first_omega_0=w0, hidden_omega_0=w0)
    model = net.cuda()

    # Define optimizer
    optim = torch.optim.Adam(lr=1e-4, params=model.parameters())
    total_steps = 1000
    steps_til_summary = 100

    psnrs = []

    for step in range(total_steps+1):
        model_output, coords = model(model_input)    
        loss = loss_functions.image_mse(model_output, coords, gt['img'])
        psnr = myutils.psnr(model_output, gt['img'])
        psnrs.append(psnr)
        if not step % steps_til_summary:
            print("Step %d, Total loss %0.6f" % (step, loss))

        optim.zero_grad()
        loss.backward()
        optim.step()

    psnr_data[str(w0)] = psnrs

myutils.plot_psnrs(psnr_data, total_steps, 'All layers $ω_0$ initialization with 8 layers', save=True, fname='psnr_w0_all_layers_1000_8_layers.png')

# 5 - SISR

### 5.1 - Define SISR config

In [ ]:
import box
import PIL
from torchvision.utils import save_image
from torchvision.transforms import Resize, Compose, ToPILImage, ToTensor, Normalize
from PIL import Image

subsets = {
    # 'bicubic_x2' : 'all',
    # 'unknown_x2' : 'all',
    # 'bicubic_x3' : 'all',
    # 'unknown_x3' : 'all',
    # 'bicubic_x4' : 'all',
    # 'unknown_x4' : 'all',
    # 'bicubic_x8' : 'all',
    # 'mild_x4' : 'all',
    # 'difficult_x4' : 'all',
    # 'wild_x4' : 'all',
    'HR_images' : 'train',
    }

# max cameramen image = 512x512
# max starfish image = 481x321
# 224: avoid VVG loss interpolation
lr_sidelength = 128 # 128, 224
hr_sidelength = 512 # 256, 512

dataset_type = 'skimage' # skimage, div2k

# Choose image type for skimage
img_type = 'cameraman' # cameraman, starfish

# Choose index for DIV2K
div2k_index = 28

# Config the network
LR = 1e-4
HIDDEN_LAYERS = 3 # 2, 3, 5, 10
HIDDEN_FEATURES = 256 # 256, 512
VERBOSE = False
TOTAL_STEPS = 500 # 500, 1000
STEPS_TIL_SUMMARY = TOTAL_STEPS*0.1

idx_recommended = [
    2,      # palma
    21,     # paesaggio
    28,     # affreschi
    33,     # carrozza
    49,     # quadro contemporaneo
    51,     # città
    68,     # donna
    82,     # paesaggio con fiume
    89,     # uomo
    109,    # asiatici
    143,    # auto
    159,    # verdura
    177,    # cammelli
    181,    # tempio
]
idx_to_sample = [div2k_index] # only the first one will be considered

### 5.2 - Prepare Dataset and Dataloader

In [ ]:
if dataset_type == 'skimage':
    lr_dataset = PoissonEqn(lr_sidelength, img_type=img_type, gradients=False, laplace=False)
    hr_dataset = PoissonEqn(hr_sidelength, img_type=img_type, gradients=False, laplace=False)
elif dataset_type == 'div2k':
    if not os.path.isdir('data/DIV2K'):
        DIV2K_loader.DIV2KImport(sets=subsets)
    lr_dataset = DIV2KImageDataset(subsets=subsets, sidelength=lr_sidelength ,transform=default_transform(lr_sidelength), idx_to_sample=idx_to_sample, with_coords=True)
    hr_dataset = DIV2KImageDataset(subsets=subsets, sidelength=hr_sidelength, transform=default_transform(hr_sidelength), idx_to_sample=idx_to_sample, with_coords=True)
else:
    raise Exception('Unexpected dataset type') 

lr_dataloader = DataLoader(lr_dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(lr_dataloader))
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

hr_dataloader = DataLoader(hr_dataset, batch_size=1, pin_memory=True, num_workers=0)

hr_grid, hr_gt = next(iter(hr_dataloader))
gt_sisr = hr_gt['img']
gt_lr_sisr = gt['img']
hr_gt = {key: value.cuda() for key, value in hr_gt.items()}
hr_grid = hr_grid.cuda()

### 5.3 - Classic SIREN

In [ ]:
# Create the network
siren_net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=HIDDEN_FEATURES)
model = siren_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output, coords = model(model_input)
    # SISR
    hr_img, _ = model(hr_grid)
    loss = loss_functions.image_mse(model_output, coords, gt['img'])

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_sisr = hr_img.cpu().detach()
psnrs_siren_sisr = psnrs
ssims_siren_sisr = ssims
psnr_siren_sisr = psnrs[-1]
ssim_siren_sisr = ssims[-1]

print('PSNR:', psnr_siren_sisr)
print('SSIM:', ssim_siren_sisr)

### 5.4 - SIREN with VGG loss

In [ ]:
input = {
    'coords' : model_input,
    'gt' : gt['img'],
    'sidelength' : lr_sidelength
}
hr_input = {
    'coords' : hr_grid,
    'gt' : hr_gt['img'],
    'sidelength' : hr_sidelength
}

# Create the network
siren_net = Siren(n_hidden_layers=HIDDEN_LAYERS, hidden_features=HIDDEN_FEATURES, with_vgg_loss=True)
# siren_net.vgg_loss.resize = False
model = siren_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output, coords, loss = model(input)
    hr_img, _, _ = model(hr_input)

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_vgg_sisr = hr_img.cpu().detach()
psnrs_siren_vgg_sisr = psnrs
ssims_siren_vgg_sisr = ssims
psnr_siren_vgg_sisr = psnrs[-1]
ssim_siren_vgg_sisr = ssims[-1]

print('PSNR:', psnr_siren_vgg_sisr)
print('SSIM:', ssim_siren_vgg_sisr)

### 5.5 - SIREN with P.E.

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
siren_net = modules.SingleBVPNet(type='sine', mode='nerf', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = siren_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']
    loss = loss_functions.image_mse(model_output, coords, gt['img'])

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_nerf_sisr = hr_img.cpu().detach()
psnrs_siren_nerf_sisr = psnrs
ssims_siren_nerf_sisr = ssims
psnr_siren_nerf_sisr = psnrs[-1]
ssim_siren_nerf_sisr = ssims[-1]

print('PSNR:', psnr_siren_nerf_sisr)
print('SSIM:', ssim_siren_nerf_sisr)

### 5.6 - SIREN with P.E. and VGG loss

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
siren_net = modules.SingleBVPNet(type='sine', mode='nerf', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = siren_net.cuda()

loss_fn = loss_functions.VGGPerceptualLoss().cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']

    # Adapt images for VGG
    _y = model_output.view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    _gt = gt['img'].view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    loss = loss_fn(_y, _gt)

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_nerf_vgg_sisr = hr_img.cpu().detach()
psnrs_siren_nerf_vgg_sisr = psnrs
ssims_siren_nerf_vgg_sisr = ssims
psnr_siren_nerf_vgg_sisr = psnrs[-1]
ssim_siren_nerf_vgg_sisr = ssims[-1]

print('PSNR:', psnr_siren_nerf_vgg_sisr)
print('SSIM:', ssim_siren_nerf_vgg_sisr)

### 5.7 - ReLU with P.E.

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
relu_net = modules.SingleBVPNet(type='relu', mode='nerf', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = relu_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']
    loss = loss_functions.image_mse(model_output, coords, gt['img'])

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
relu_sisr = hr_img.cpu().detach()
psnrs_relu_sisr = psnrs
ssims_relu_sisr = ssims
psnr_relu_sisr = psnrs[-1]
ssim_relu_sisr = ssims[-1]

print('PSNR:', psnr_relu_sisr)
print('SSIM:', ssim_relu_sisr)

### 5.8 - Bicubic

In [ ]:
to_hr = Compose([
    Resize(hr_sidelength, Image.BICUBIC),
    ToTensor()
])

bicubic_input = gt['img'].clone().cpu().detach().view(lr_sidelength, lr_sidelength).numpy()
bicubic_input = ToPILImage()(bicubic_input)

hr_img = to_hr(bicubic_input) # SISR

_, axes = plt.subplots(1, 3, figsize=(18,6))
axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
plt.show()

bicubic_sisr = hr_img.cpu().detach()

psnr_bicubic_sisr = myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength)
ssim_bicubic_sisr = myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength)

psnrs_bicubic_sisr = [psnr_bicubic_sisr for i in range(TOTAL_STEPS+1)]
ssims_bicubic_sisr = [ssim_bicubic_sisr for i in range(TOTAL_STEPS+1)]

print('PSNR:', psnr_bicubic_sisr)
print('SSIM:', ssim_bicubic_sisr)

### 5.9 - SRGAN

In [ ]:
!rm -r '/content/mydataset'
os.mkdir('/content/mydataset')
os.mkdir('/content/mydataset/mydataset')
os.mkdir('/content/mydataset/mydataset/data')
os.mkdir('/content/mydataset/mydataset/data/images')
os.mkdir('/content/mydataset/mydataset/data/images/train')
os.mkdir('/content/mydataset/mydataset/data/groundTruth')
os.mkdir('/content/mydataset/mydataset/data/groundTruth/train')

image = hr_gt['img'].clone().cpu().detach().view(hr_sidelength, hr_sidelength)
image = (image +1) /2.

name = 'test_img'
fname = name + '.png'

save_image(image, '/content/mydataset/mydataset/data/groundTruth/train/' + fname)
save_image(image, '/content/mydataset/mydataset/data/images/train/' + fname)

!CUDA_VISIBLE_DEVICES=0 python ./SRGAN-PyTorch/test.py --option /content/siren/src/srgan/config_test.json

transform = Compose([
    Resize(hr_sidelength),
    ToTensor(),
    Normalize(torch.Tensor([0.5]), torch.Tensor([0.5])) # image = (image - mean) / std, output values in [-1., +1.]
])

srgan_img = Image.open('/content/results/SRGAN_x4/test_images/mydataset/' + name + '/sr.png')
srgan_img = srgan_img.convert('L')
srgan_img = transform(srgan_img)

_, axes = plt.subplots(1, 3, figsize=(18,6))
axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
axes[1].imshow(srgan_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
plt.show()

srgan_sisr = srgan_img

psnr_srgan_sisr = myutils.img_psnr(srgan_img, hr_gt['img'], hr_sidelength)
ssim_srgan_sisr = myutils.img_ssim(srgan_img, hr_gt['img'], hr_sidelength)

psnrs_srgan_sisr = [psnr_srgan_sisr for i in range(TOTAL_STEPS+1)]
ssims_srgan_sisr = [ssim_srgan_sisr for i in range(TOTAL_STEPS+1)]

print('PSNR:', psnr_srgan_sisr)
print('SSIM:', ssim_srgan_sisr)

### 5.10 - Collect and compare results

In [ ]:
psnr_data = {
    'SIREN' : psnrs_siren_sisr,
    'SIREN VGG' : psnrs_siren_vgg_sisr,
    'SIREN P.E.' : psnrs_siren_nerf_sisr,
    'SIREN P.E. VGG' : psnrs_siren_nerf_vgg_sisr,
    'ReLU P.E.' : psnrs_relu_sisr,
    'bicubic' : psnrs_bicubic_sisr,
    'SRGAN' : psnrs_srgan_sisr,
    }
ssim_data = {
    'SIREN' : ssims_siren_sisr,
    'SIREN VGG' : ssims_siren_vgg_sisr,
    'SIREN P.E.' : ssims_siren_nerf_sisr,
    'SIREN P.E. VGG' : ssims_siren_nerf_vgg_sisr,
    'ReLU P.E.' : ssims_relu_sisr,
    'bicubic' : ssims_bicubic_sisr,
    'SRGAN' : ssims_srgan_sisr,
    }

myutils.plot_psnr_and_ssim(psnr_data, ssim_data, total_steps, save=True, fname='psnr_and_ssim_sisr.png')

# images = {
#     'GT' : gt_sisr,
#     'SIREN' : siren_sisr,
#     'SIREN VGG' : siren_vgg_sisr,
#     'SIREN P.E.' : siren_nerf_sisr,
#     # 'SIREN P.E. VGG' : siren_nerf_vgg_sisr,
#     'ReLU P.E.' : relu_sisr,
#     'Bicubic' : bicubic_sisr,
#     'SRGAN' : srgan_sisr,
#     }
images = {
    'LR GT' : gt_lr_sisr,
    'HR GT' : gt_sisr,
    'SIREN' : siren_sisr,
    'SIREN P.E. VGG' : siren_vgg_sisr,
    'SIREN P.E.' : siren_nerf_sisr,
    'SIREN P.E. VGG' : siren_nerf_vgg_sisr,
    'ReLU P.E.' : relu_sisr,
    'Bicubic' : bicubic_sisr,
    'SRGAN' : srgan_sisr,
    }

def print_sisr_grid(images, hr_sidelength=512, lr_sidelength=512, figsize=(20, 10), textsize=24, fname='sisr_grid.png'):
    ''' Create custom figure with grid (GT, SIREN, SIREN VGG, ReLU-Nerf, Bicubic, SRGAN)'''

    fig = plt.figure(constrained_layout=False, figsize=figsize)
    gs = fig.add_gridspec(2, 4, wspace=0, hspace=0.3)

    for i, (key, value) in enumerate(images.items()):
        if i < 4:
            a1 = fig.add_subplot(gs[0, i])
        else:
            a1 = fig.add_subplot(gs[1, i-4])
        if key == 'LR GT':
            a1.imshow(value.cpu().view(lr_sidelength, lr_sidelength).detach().numpy(), cmap='gray')
        else:
            a1.imshow(value.cpu().view(hr_sidelength, hr_sidelength).detach().numpy(), cmap='gray')
        a1.set_xticks([])
        a1.set_yticks([])
        a1.set_title(key, fontsize=textsize)
    plt.savefig(fname=fname, bbox_inches='tight')
    plt.show()

print_sisr_grid(images, hr_sidelength=hr_sidelength, lr_sidelength=lr_sidelength)

# myutils.print_sisr_grid(images, sidelength=hr_sidelength)

print('SISR %dx%d to %dx%d' % (lr_sidelength, lr_sidelength, hr_sidelength, hr_sidelength))
print('hidden layers: %d' % (HIDDEN_LAYERS))
print('hidden features: %d' % (HIDDEN_FEATURES))
print('total steps: %d' % (TOTAL_STEPS))
print('\n')

print('PSNR:')
for key, val in psnr_data.items():
    print('\t' + key + ': %0.6f' % (val[-1]))
print('\n')
print('SSIM:')
for key, val in ssim_data.items():
    print('\t' + key + ': %0.6f' % (val[-1]))

# 6 - Supplementary materials

## 6.1 - Compare ReLU classic, VGG and P.E.

### 6.1.1 - Define config

In [ ]:
import box
import PIL
from torchvision.utils import save_image
from torchvision.transforms import Resize, Compose, ToPILImage, ToTensor, Normalize
from PIL import Image

subsets = {
    # 'bicubic_x2' : 'all',
    # 'unknown_x2' : 'all',
    # 'bicubic_x3' : 'all',
    # 'unknown_x3' : 'all',
    # 'bicubic_x4' : 'all',
    # 'unknown_x4' : 'all',
    # 'bicubic_x8' : 'all',
    # 'mild_x4' : 'all',
    # 'difficult_x4' : 'all',
    # 'wild_x4' : 'all',
    'HR_images' : 'train',
    }

# max cameramen image = 512x512
# max starfish image = 481x321
# 224: avoid VVG loss interpolation
lr_sidelength = 128 # 64, 128, 224, 256
hr_sidelength = 512 # 256, 512

dataset_type = 'div2k' # skimage, div2k

# Choose image type for skimage
img_type = 'cameraman' # cameraman, starfish

# Choose index for DIV2K
div2k_index = 143

# Config the network
LR = 1e-4
HIDDEN_LAYERS = 3 # 2, 3, 5, 10
HIDDEN_FEATURES = 256 # 256. 512
VERBOSE = False
TOTAL_STEPS = 1000 # 500, 1000
STEPS_TIL_SUMMARY = TOTAL_STEPS*0.1

idx_recommended = [
    2,      # palma
    21,     # paesaggio
    28,     # affreschi
    33,     # carrozza
    49,     # quadro contemporaneo
    51,     # città
    68,     # donna
    82,     # paesaggio con fiume
    89,     # uomo
    109,    # asiatici
    143,    # auto
    159,    # verdura
    177,    # cammelli
    181,    # tempio
]
idx_to_sample = [div2k_index] # only the first one will be considered

### 6.1.2 - Prepare Dataset and Dataloader

In [ ]:
if dataset_type == 'skimage':
    lr_dataset = PoissonEqn(lr_sidelength, img_type=img_type, gradients=False, laplace=False)
    hr_dataset = PoissonEqn(hr_sidelength, img_type=img_type, gradients=False, laplace=False)
elif dataset_type == 'div2k':
    if not os.path.isdir('data/DIV2K'):
        DIV2K_loader.DIV2KImport(sets=subsets)
    lr_dataset = DIV2KImageDataset(subsets=subsets, sidelength=lr_sidelength ,transform=default_transform(lr_sidelength), idx_to_sample=idx_to_sample, with_coords=True)
    hr_dataset = DIV2KImageDataset(subsets=subsets, sidelength=hr_sidelength, transform=default_transform(hr_sidelength), idx_to_sample=idx_to_sample, with_coords=True)
else:
    raise Exception('Unexpected dataset type') 

lr_dataloader = DataLoader(lr_dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(lr_dataloader))
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

hr_dataloader = DataLoader(hr_dataset, batch_size=1, pin_memory=True, num_workers=0)

hr_grid, hr_gt = next(iter(hr_dataloader))
gt_sisr = hr_gt['img']
hr_gt = {key: value.cuda() for key, value in hr_gt.items()}
hr_grid = hr_grid.cuda()

### 6.1.3 - Classic ReLU

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
relu_net = modules.SingleBVPNet(type='relu', mode='mlp', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = relu_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']
    loss = loss_functions.image_mse(model_output, coords, gt['img'])

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
relu_sisr = hr_img.cpu().detach()
psnrs_relu_sisr = psnrs
ssims_relu_sisr = ssims
psnr_relu_sisr = psnrs[-1]
ssim_relu_sisr = ssims[-1]

print('PSNR:', psnr_relu_sisr)
print('SSIM:', ssim_relu_sisr)

### 6.1.4 - ReLU with VGG loss

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
relu_net = modules.SingleBVPNet(type='relu', mode='mlp', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = relu_net.cuda()

loss_fn = loss_functions.VGGPerceptualLoss().cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']

    # Adapt images for VGG
    _y = model_output.view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    _gt = gt['img'].view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    loss = loss_fn(_y, _gt)

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
relu_vgg_sisr = hr_img.cpu().detach()
psnrs_relu_vgg_sisr = psnrs
ssims_relu_vgg_sisr = ssims
psnr_relu_vgg_sisr = psnrs[-1]
ssim_relu_vgg_sisr = ssims[-1]

print('PSNR:', psnr_relu_vgg_sisr)
print('SSIM:', ssim_relu_vgg_sisr)

### 6.1.5 - ReLU with P.E.

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
relu_net = modules.SingleBVPNet(type='relu', mode='nerf', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = relu_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']
    loss = loss_functions.image_mse(model_output, coords, gt['img'])

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
relu_nerf_sisr = hr_img.cpu().detach()
psnrs_relu_nerf_sisr = psnrs
ssims_relu_nerf_sisr = ssims
psnr_relu_nerf_sisr = psnrs[-1]
ssim_relu_nerf_sisr = ssims[-1]

print('PSNR:', psnr_relu_nerf_sisr)
print('SSIM:', ssim_relu_nerf_sisr)

### 6.1.6 - ReLU with P.E. and VGG loss

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
relu_net = modules.SingleBVPNet(type='relu', mode='nerf', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = relu_net.cuda()

loss_fn = loss_functions.VGGPerceptualLoss().cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']

    # Adapt images for VGG
    _y = model_output.view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    _gt = gt['img'].view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    loss = loss_fn(_y, _gt)

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
relu_nerf_vgg_sisr = hr_img.cpu().detach()
psnrs_relu_nerf_vgg_sisr = psnrs
ssims_relu_nerf_vgg_sisr = ssims
psnr_relu_nerf_vgg_sisr = psnrs[-1]
ssim_relu_nerf_vgg_sisr = ssims[-1]

print('PSNR:', psnr_relu_nerf_vgg_sisr)
print('SSIM:', ssim_relu_nerf_vgg_sisr)

### 6.1.7 - Collect and compare results

In [ ]:
psnr_data = {
    'ReLU' : psnrs_relu_sisr,
    'ReLU VGG' : psnrs_relu_vgg_sisr,
    'ReLU P.E.' : psnrs_relu_nerf_sisr,
    'ReLU P.E. VGG' : psnrs_relu_nerf_vgg_sisr,
    }
ssim_data = {
    'ReLU' : ssims_relu_sisr,
    'ReLU VGG' : ssims_relu_vgg_sisr,
    'ReLU P.E.' : ssims_relu_nerf_sisr,
    'ReLU P.E. VGG' : ssims_relu_nerf_vgg_sisr,
    }

myutils.plot_psnr_and_ssim(psnr_data, ssim_data, total_steps, save=True, fname='psnr_and_ssim_sisr_relu.png')

images = {
    'GT' : gt_sisr,
    'ReLU' : relu_sisr,
    'ReLU VGG' : relu_vgg_sisr,
    'ReLU P.E.' : relu_nerf_sisr,
    'ReLU P.E. VGG' : relu_nerf_vgg_sisr,
    }

def print_sisr_grid(images, sidelength=512, figsize=(20, 4), textsize=24, fname='sisr_grid_relu.png'):
    ''' Create custom figure with grid (GT, SIREN, SIREN VGG, ReLU-Nerf, Bicubic, SRGAN)'''

    fig = plt.figure(constrained_layout=False, figsize=figsize)
    gs = fig.add_gridspec(1, len(images), wspace=0, hspace=0)

    for i, (key, value) in enumerate(images.items()):
        a1 = fig.add_subplot(gs[0, i])
        a1.imshow(value.cpu().view(sidelength, sidelength).detach().numpy(), cmap='gray')
        a1.set_xticks([])
        a1.set_yticks([])
        a1.set_title(key, fontsize=textsize)
    plt.savefig(fname=fname, bbox_inches='tight')
    plt.show()

print_sisr_grid(images, sidelength=hr_sidelength)

print('SISR %dx%d to %dx%d' % (lr_sidelength, lr_sidelength, hr_sidelength, hr_sidelength))
print('hidden layers: %d' % (HIDDEN_LAYERS))
print('hidden features: %d' % (HIDDEN_FEATURES))
print('total steps: %d' % (TOTAL_STEPS))
print('\n')

print('PSNR:')
for key, val in psnr_data.items():
    print('\t' + key + ': %0.6f' % (val[-1]))
print('\n')
print('SSIM:')
for key, val in ssim_data.items():
    print('\t' + key + ': %0.6f' % (val[-1]))

## 6.2 - Compare SIREN classic, VGG and P.E.

### 6.2.1 - Define config

In [ ]:
import box
import PIL
from torchvision.utils import save_image
from torchvision.transforms import Resize, Compose, ToPILImage, ToTensor, Normalize
from PIL import Image

subsets = {
    # 'bicubic_x2' : 'all',
    # 'unknown_x2' : 'all',
    # 'bicubic_x3' : 'all',
    # 'unknown_x3' : 'all',
    # 'bicubic_x4' : 'all',
    # 'unknown_x4' : 'all',
    # 'bicubic_x8' : 'all',
    # 'mild_x4' : 'all',
    # 'difficult_x4' : 'all',
    # 'wild_x4' : 'all',
    'HR_images' : 'train',
    }

# max cameramen image = 512x512
# max starfish image = 481x321
# 224: avoid VVG loss interpolation
lr_sidelength = 128 # 128, 224, 256
hr_sidelength = 512 # 256, 512

dataset_type = 'div2k' # skimage, div2k

# Choose image type for skimage
img_type = 'cameraman' # cameraman, starfish

# Choose index for DIV2K
div2k_index = 143

# Config the network
LR = 1e-4
HIDDEN_LAYERS = 3 # 2, 3, 5, 10
HIDDEN_FEATURES = 256 # 256. 512
VERBOSE = False
TOTAL_STEPS = 100 # 500, 1000
STEPS_TIL_SUMMARY = TOTAL_STEPS*0.1

idx_recommended = [
    2,      # palma
    21,     # paesaggio
    28,     # affreschi
    33,     # carrozza
    49,     # quadro contemporaneo
    51,     # città
    68,     # donna
    82,     # paesaggio con fiume
    89,     # uomo
    109,    # asiatici
    143,    # auto
    159,    # verdura
    177,    # cammelli
    181,    # tempio
]
idx_to_sample = [div2k_index] # only the first one will be considered

### 6.1.2 - Prepare Dataset and Dataloader

In [ ]:
if dataset_type == 'skimage':
    lr_dataset = PoissonEqn(lr_sidelength, img_type=img_type, gradients=False, laplace=False)
    hr_dataset = PoissonEqn(hr_sidelength, img_type=img_type, gradients=False, laplace=False)
elif dataset_type == 'div2k':
    if not os.path.isdir('data/DIV2K'):
        DIV2K_loader.DIV2KImport(sets=subsets)
    lr_dataset = DIV2KImageDataset(subsets=subsets, sidelength=lr_sidelength ,transform=default_transform(lr_sidelength), idx_to_sample=idx_to_sample, with_coords=True)
    hr_dataset = DIV2KImageDataset(subsets=subsets, sidelength=hr_sidelength, transform=default_transform(hr_sidelength), idx_to_sample=idx_to_sample, with_coords=True)
else:
    raise Exception('Unexpected dataset type') 

lr_dataloader = DataLoader(lr_dataset, batch_size=1, pin_memory=True, num_workers=0)

model_input, gt = next(iter(lr_dataloader))
gt = {key: value.cuda() for key, value in gt.items()}
model_input = model_input.cuda()

hr_dataloader = DataLoader(hr_dataset, batch_size=1, pin_memory=True, num_workers=0)

hr_grid, hr_gt = next(iter(hr_dataloader))
gt_sisr = hr_gt['img']
hr_gt = {key: value.cuda() for key, value in hr_gt.items()}
hr_grid = hr_grid.cuda()

### 6.2.3 - Classic SIREN

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
siren_net = modules.SingleBVPNet(type='sine', mode='mlp', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = siren_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']
    loss = loss_functions.image_mse(model_output, coords, gt['img'])

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_sisr = hr_img.cpu().detach()
psnrs_siren_sisr = psnrs
ssims_siren_sisr = ssims
psnr_siren_sisr = psnrs[-1]
ssim_siren_sisr = ssims[-1]

print('PSNR:', psnr_siren_sisr)
print('SSIM:', ssim_siren_sisr)

### 6.2.4 - SIREN with VGG loss

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
siren_net = modules.SingleBVPNet(type='sine', mode='mlp', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = siren_net.cuda()

loss_fn = loss_functions.VGGPerceptualLoss().cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']

    # Adapt images for VGG
    _y = model_output.view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    _gt = gt['img'].view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    loss = loss_fn(_y, _gt)

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_vgg_sisr = hr_img.cpu().detach()
psnrs_siren_vgg_sisr = psnrs
ssims_siren_vgg_sisr = ssims
psnr_siren_vgg_sisr = psnrs[-1]
ssim_siren_vgg_sisr = ssims[-1]

print('PSNR:', psnr_siren_vgg_sisr)
print('SSIM:', ssim_siren_vgg_sisr)

### 6.2.5 - SIREN with P.E.

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
siren_net = modules.SingleBVPNet(type='sine', mode='nerf', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = siren_net.cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']
    loss = loss_functions.image_mse(model_output, coords, gt['img'])

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_nerf_sisr = hr_img.cpu().detach()
psnrs_siren_nerf_sisr = psnrs
ssims_siren_nerf_sisr = ssims
psnr_siren_nerf_sisr = psnrs[-1]
ssim_siren_nerf_sisr = ssims[-1]

print('PSNR:', psnr_siren_nerf_sisr)
print('SSIM:', ssim_siren_nerf_sisr)

### 6.2.6 - SIREN with P.E. and VGG loss

In [ ]:
sys.path.append('siren/src/include/siren')
import modules

input = {'coords' : model_input}
hr_input = {'coords' : hr_grid}

# Create the network
siren_net = modules.SingleBVPNet(type='relu', mode='nerf', hidden_features=HIDDEN_FEATURES, num_hidden_layers=HIDDEN_LAYERS, sidelength=lr_sidelength)
model = siren_net.cuda()

loss_fn = loss_functions.VGGPerceptualLoss().cuda()

# Define optimizer
optim = torch.optim.Adam(lr=LR, params=model.parameters())

total_steps = TOTAL_STEPS
steps_til_summary = STEPS_TIL_SUMMARY
verbose = VERBOSE # True, False

psnrs = []
ssims = []

for step in range(total_steps+1):
    model_output = model(input)    
    coords = model_output['model_in']
    model_output = model_output['model_out']
    hr_img = model(hr_input)    
    hr_img = hr_img['model_out']

    # Adapt images for VGG
    _y = model_output.view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    _gt = gt['img'].view(lr_sidelength, lr_sidelength).unsqueeze(0).unsqueeze(0)
    loss = loss_fn(_y, _gt)

    psnrs.append(myutils.img_psnr(hr_img, hr_gt['img'], hr_sidelength))
    ssims.append(myutils.img_ssim(hr_img, hr_gt['img'], hr_sidelength))
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        print('GPU Memory occupation: %0.1f / %0.1f GB' % (torch.cuda.memory_allocated(0) / (1024 ** 3), torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))
        
        if verbose or step == total_steps:
            _, axes = plt.subplots(1, 3, figsize=(18,6))
            axes[0].imshow(gt['img'].cpu().view(lr_sidelength, lr_sidelength).detach().numpy())
            axes[0].set_title('GT '+ str(lr_sidelength) + 'x' + str(lr_sidelength))
            axes[1].imshow(hr_img.cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[1].set_title('SISR ' + str(lr_sidelength) + ' to ' + str(hr_sidelength))
            axes[2].imshow(hr_gt['img'].cpu().view(hr_sidelength, hr_sidelength).detach().numpy())
            axes[2].set_title('GT ' + str(hr_sidelength) + 'x' + str(hr_sidelength))
            plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

# Collect data to compare results
siren_nerf_vgg_sisr = hr_img.cpu().detach()
psnrs_siren_nerf_vgg_sisr = psnrs
ssims_siren_nerf_vgg_sisr = ssims
psnr_siren_nerf_vgg_sisr = psnrs[-1]
ssim_siren_nerf_vgg_sisr = ssims[-1]

print('PSNR:', psnr_siren_nerf_vgg_sisr)
print('SSIM:', ssim_siren_nerf_vgg_sisr)

### 6.2.7 - Collect and compare results

In [ ]:
psnr_data = {
    'SIREN' : psnrs_siren_sisr,
    'SIREN VGG' : psnrs_siren_vgg_sisr,
    'SIREN P.E.' : psnrs_siren_nerf_sisr,
    'SIREN P.E. VGG' : psnrs_siren_nerf_vgg_sisr,
    }
ssim_data = {
    'SIREN' : ssims_siren_sisr,
    'SIREN VGG' : ssims_siren_vgg_sisr,
    'SIREN P.E.' : ssims_siren_nerf_sisr,
    'SIREN P.E. VGG' : ssims_siren_nerf_vgg_sisr,
    }

myutils.plot_psnr_and_ssim(psnr_data, ssim_data, total_steps, save=True, fname='psnr_and_ssim_sisr_siren.png')

images = {
    'GT' : gt_sisr,
    'SIREN' : siren_sisr,
    'SIREN VGG' : siren_vgg_sisr,
    'SIREN P.E.' : siren_nerf_sisr,
    'SIREN P.E. VGG' : siren_nerf_vgg_sisr,
    }

def print_sisr_grid(images, sidelength=512, figsize=(20, 4), textsize=24, fname='sisr_grid_siren.png'):
    fig = plt.figure(constrained_layout=False, figsize=figsize)
    gs = fig.add_gridspec(1, len(images), wspace=0, hspace=0)

    for i, (key, value) in enumerate(images.items()):
        a1 = fig.add_subplot(gs[0, i])
        a1.imshow(value.cpu().view(sidelength, sidelength).detach().numpy(), cmap='gray')
        a1.set_xticks([])
        a1.set_yticks([])
        a1.set_title(key, fontsize=textsize)
    plt.savefig(fname=fname, bbox_inches='tight')
    plt.show()

print_sisr_grid(images, sidelength=hr_sidelength)

print('SISR %dx%d to %dx%d' % (lr_sidelength, lr_sidelength, hr_sidelength, hr_sidelength))
print('hidden layers: %d' % (HIDDEN_LAYERS))
print('hidden features: %d' % (HIDDEN_FEATURES))
print('total steps: %d' % (TOTAL_STEPS))
print('\n')

print('PSNR:')
for key, val in psnr_data.items():
    print('\t' + key + ': %0.6f' % (val[-1]))
print('\n')
print('SSIM:')
for key, val in ssim_data.items():
    print('\t' + key + ': %0.6f' % (val[-1]))